# 🎨 Image Style Transfer to Ghibli, Pixar, Anime, and More
This notebook lets users upload a photo and convert it to a stylized art image (e.g., Ghibli, Pixar, Anime, etc.) using open-source models from HuggingFace.

**Goal**: Take a photo and output it in a new art style.
- Output format: PNG
- Privacy: No external APIs, runs locally in Colab
- Interface: Streamlit Web App (deployment section below)

Supported Styles:
- Ghibli
- Pixar
- Lego
- Watercolor
- Simpson
- Disney
- Chibi
- Comic
- Anime
- Powerpuff Girls


In [ ]:
# Cell 1: Install required libraries
!pip install diffusers transformers accelerate --quiet
!pip install xformers --quiet
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 --quiet

In [ ]:
# Cell 2: Import libraries
import torch
from PIL import Image
from diffusers import StableDiffusionImg2ImgPipeline
import requests
from io import BytesIO
import os

In [ ]:
# Cell 3: Load the model from HuggingFace (no API key needed)
# We use a general Img2Img model for style prompts
model_id = "CompVis/stable-diffusion-v1-4"
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    revision="fp16",
    use_auth_token=False
).to("cuda")
pipe.enable_xformers_memory_efficient_attention()

In [ ]:
# Cell 4: Upload an image
from google.colab import files
uploaded = files.upload()
image_path = next(iter(uploaded))
init_image = Image.open(image_path).convert("RGB")
init_image = init_image.resize((512, 512))
init_image.save("original.png")

In [ ]:
# Cell 5: Define style prompts
style_prompts = {
    "Ghibli": "A portrait in Studio Ghibli anime style",
    "Pixar": "Pixar-style 3D render of a person",
    "Lego": "Lego figure style image of a person",
    "Watercolor": "Watercolor painting of a person",
    "Simpson": "A character in the Simpsons cartoon style",
    "Disney": "Disney animated character portrait",
    "Chibi": "Cute Chibi anime character portrait",
    "Comic": "Comic book style portrait illustration",
    "Anime": "Japanese anime style character",
    "Powerpuff": "Powerpuff Girls character portrait"
}

In [ ]:
# Cell 6: Generate stylized image
style = "Ghibli"  # Change this manually to another style from the dict
prompt = style_prompts[style]
image = pipe(prompt=prompt, image=init_image, strength=0.75, guidance_scale=7.5).images[0]
output_filename = f"stylized_{style}.png"
image.save(output_filename)
image.show()

In [ ]:
# Cell 7: Save model (not strictly required, but for completeness)
import pickle
with open("style_transfer_pipe.pkl", "wb") as f:
    pickle.dump(pipe, f)